In [ ]:
from copy import deepcopy
import math
import sys

import matplotlib.pylab as plt
import numpy as np

%reload_ext autoreload
%autoreload 2

#%matplotlib inline
%matplotlib notebook

sys.path.append('../crazyflie-audio/python/')

In [ ]:
from bin_selection import select_frequencies
from signals import generate_signal, amplify_signal
from scipy.spatial.transform import Rotation

def rotate_mics(mics, orientation_deg=0):
    """
    :param mics: mic positions (n_mics, 2)
    :return mics_rotated: (n_mics, 2)
    """
    rot = Rotation.from_euler('z', orientation_deg, degrees=True)
    R = rot.as_matrix() # 3x3
    mics_aug = np.c_[mics, np.ones(mics.shape[0])].T # 3 x 4
    mics_rotated = R.dot(mics_aug)[:2, :] # 2 x 4
    return mics_rotated.T

def plot_spectrum(spectrum, degree=0):
    plt.figure()
    if spectrum.shape[0] == 1:
        plt.plot(angles, np.log10(spectrum[0]))
    else:
        plt.pcolormesh(angles, frequencies, np.log10(spectrum))
    plt.axvline(degree, color='red')
    plt.xlabel('angle [deg]')
    plt.ylabel('frequency [Hz]')
    
def from_0_to_2pi(angle):
    angle = (angle + np.pi) % (2 * np.pi) - np.pi # -pi to pi
    if (type(angle) == float) or (angle.ndim == 0):
        angle = angle + 2 * np.pi if angle < 0 else angle
    else:
        angle[angle < 0] += 2 * np.pi
    return angle

def from_0_to_360(angle_deg):
    return 180 / np.pi * from_0_to_2pi(angle_deg / 180 * np.pi)

In [ ]:
def generate_signals_pyroom(source_signal, mics_rotated, time, noise=0, ax=None):
    import pyroomacoustics as pra
    room = pra.AnechoicRoom(fs=Fs, dim=2)
    room.add_source(source, signal=source_signal, name=f"source")

    beam_former = pra.Beamformer(mics_rotated, Fs)
    room.add_microphone_array(beam_former)
    room.simulate()
    
    time_idx = int(round(time * Fs))
    print(time, Fs, time_idx)
    print('error:', time_idx / Fs, time, (time_idx / Fs) - time)
    
    signals = deepcopy(room.mic_array.signals)
    if noise > 0:
        signals += np.random.normal(scale=noise, size=signals.shape)
    print('shifted signals by', time_idx, signals.shape)
    
    if ax is not None:
        for i in range(signals.shape[0]):
            ax.plot(signals[i])
        ax.axvline(x=time_idx)
    return signals[:, time_idx:]

def generate_signals(source_signal, mics_rotated, time, noise=0, ax=None):
    from algos_beamforming import get_mic_delays
    from constants import SPEED_OF_SOUND

    n_mics = mics_rotated.shape[0]
    print(mics_rotated.shape)
    print('mic 0:', mics_rotated[0])
    print('mic 1:', mics_rotated[1])

    delays_relative = get_mic_delays(mics_rotated, gt_angle_rad)
    delays = np.linalg.norm(mics_rotated[0] - source)/SPEED_OF_SOUND + delays_relative

    times = np.arange(0, duration, step=1/Fs) # n_times
    signals = np.sin(2*np.pi*frequency_hz*(times[None, :] - delays[:, None] - time)) # n_mics x n_times
    signals[times[None, :] < delays[:, None]] = 0.0

    if noise > 0:
        signals += np.random.normal(scale=noise, size=signals.shape)

    if ax is not None:
        for i in range(signals.shape[0]):
            ax.plot(signals[i])
        #ax.set_xlim(0, 100)
    return signals

# 1. Geometrical setup

In [ ]:
from mic_array import get_square_array, get_uniform_array

baseline = 0.108  # meters, square side of mic array
gt_distance = 10  # meters, distance of source
add_noise_mics_positions = 1e-3  # noise added to mic positions
add_noise_times = 1e-6 # noise added to recording times

#mics_drone = get_square_array(baseline=baseline, delta=0) # 4 x 2
mics_drone = get_uniform_array(2, baseline=baseline) # 4 x 2

gt_angle_deg = 90
gt_angle_rad = gt_angle_deg * np.pi / 180.0
source = gt_distance * np.array([np.cos(gt_angle_rad), np.sin(gt_angle_rad)])

plt.figure()
plt.scatter(*mics_drone.T)
plt.scatter(*source)
plt.axis('equal')

plt.figure()
plt.scatter(*mics_drone.T)
plt.axis('equal')

# 2. Simulate signals at mics

In [ ]:
Fs = 44100  # Hz, sampling frequency
n_buffer = 2048
duration = 100  # seconds, should be long enough to acocunt for delays and full movement
np.random.seed(1)
noise = 0.0

frequencies = np.fft.rfftfreq(n_buffer, 1/Fs)

signal_type = "mono"; 
frequency_desired = 600
indices = [np.argmin(np.abs(frequencies - frequency_desired))]
print(indices)
frequency_hz = frequencies[indices]

print(frequency_hz)

source_signal = generate_signal(Fs, 
                                signal_type=signal_type, 
                                duration_sec=duration, 
                                frequency_hz=frequency_hz, 
                                noise=noise)
source_signal = amplify_signal(source_signal, target_dB=-10)

In [ ]:
# create one long source signal
angular_velocity_deg = 20 # deg/sec, velocity of drone
time_index = 1000 # idx where signal is non-zero for all positions, found heuristically

#degrees = [0, 0]
#degrees = range(10)
#degrees = np.arange(0, 360, 45)
#degrees = [0, 85] #, -40] 
degrees = [0, 20, 45]

if angular_velocity_deg != 0:
    times_list = [(d-degrees[0])/angular_velocity_deg for d in degrees] # seconds
else:
    times_list = [0] * len(degrees)
print('recording times in seconds:', times_list)

assert duration > max(times_list)

np.random.seed(1)
mics_list = [rotate_mics(mics_drone, orientation_deg=degree) for degree in degrees]

plt.figure()
for degree, mics in zip(degrees, mics_list):
    for j in range(mics.shape[0]):
        plt.scatter(*mics[j], label=f'rot {degree}, mic{j}')
plt.axis('equal')
plt.legend(bbox_to_anchor=[1, 1], loc='upper left')
pass

In [ ]:
signals_list = []

%matplotlib notebook
for mics, time in zip(mics_list, times_list):
    
    fig, ax = plt.subplots()
    signals_received =  generate_signals(source_signal, mics, time, noise=noise, ax=ax) 
    ax.set_xlim(800, 1400)
    ax.legend([f"mic{i}" for i in range(mics.shape[0])])
    ax.set_title(f'signals at time {time:.3f}')
    
    signals_list.append(signals_received)
buffers = [signals_this[:, time_index:time_index + n_buffer] for signals_this in signals_list] 

if add_noise_mics_positions > 0:
    mics_drone += np.random.normal(scale=add_noise_mics_positions, size=mics_drone.shape)
if add_noise_times > 0:
    times_list = [t + np.random.normal(scale=add_noise_times) for t in times_list]

# sanity check of signal creation

In [ ]:
fig, ax = plt.subplots()
for mic in mics_drone:
    ax.scatter(*mic)
ax.scatter(*source)
ax.axis('equal')
    
fig, ax = plt.subplots()
generate_signals_pyroom(source_signal, mics_list[-1].T, time=1.0, noise=0, ax=ax)
ax.set_title('pyroom')
ax.set_xlim(800, 1000)

fig, ax = plt.subplots()
generate_signals(source_signal, mics_list[-1], time=1.0, noise=0, ax=ax)
ax.set_title('ours')
ax.set_xlim(800, 1000)

# 3. Create "real" multi-mic array

In [ ]:
mics_array = np.concatenate([*mics_list])
signals_multimic = generate_signals(source_signal, mics_array, time=0)
buffer_multimic = signals_multimic[:, time_index:time_index + n_buffer]

fig, axs = plt.subplots(buffer_multimic.shape[0], sharex=True)
for i in range(buffer_multimic.shape[0]): # n_mics
    label=f"mic{i}"
    axs[i].plot(np.arange(buffer_multimic.shape[1])/Fs, buffer_multimic[i], label=label)
    axs[i].set_title(label)
    axs[i].set_xlim(0, 0.002)
plt.suptitle('multi-mic signals')

In [ ]:
fig, axs = plt.subplots(buffer_multimic.shape[0], sharex=True)
counter = 0
for j, (degree, buffer, time) in enumerate(zip(degrees, buffers, times_list)):
    for i in range(buffer.shape[0]): # n_mics
        phase_shift = from_0_to_2pi(- 2 * np.pi * time * frequency_hz) * 180 / np.pi
        time_shift = phase_shift / (360 * frequency_hz)
        print("phase shift in degrees", phase_shift)
        print("phase shift in time", time_shift)
        label = f"rot {degree} time {time:.3f} mic{i}"
        axs[counter].axvline(time_shift)
        axs[counter].plot(np.arange(buffer.shape[1])/Fs, buffer[i], label=label)
        axs[counter].set_title(label)
        axs[counter].set_xlim(0, 0.002)
        counter += 1
plt.suptitle('recorded signals without delay correction')

# 4. DOA estimation

# "real" spectrum

In [ ]:
from audio_stack.beam_former import BeamFormer

beam_former = BeamFormer(mic_positions=mics_array)
angles = beam_former.theta_scan * 180 / np.pi

signals_f_multimic = np.fft.rfft(buffer_multimic).T

signals_f_multimic = signals_f_multimic[indices, :]
frequencies = frequencies[indices]

R_multimic = beam_former.get_correlation(signals_f_multimic)
print(np.angle(R_multimic))
spectrum_multimic = beam_former.get_mvdr_spectrum(R_multimic, frequencies)
plot_spectrum(spectrum_multimic, degree=gt_angle_deg)

# combine spectra

In [ ]:
beam_former = BeamFormer(mic_positions=mics_drone)

# calculate individual spectra for each pose
signals_f_list = [np.fft.rfft(buffer).T for buffer in buffers] # n_frequences x 4
signals_f_list = [sig_f[indices, :] for sig_f in signals_f_list]
    
Rs = [beam_former.get_correlation(sig_f) for sig_f in signals_f_list]
#spectra = [beam_former.get_das_spectrum(R, frequencies) for R in Rs]
spectra = [beam_former.get_mvdr_spectrum(R, frequencies) for R in Rs]

# combine the individual spectra, accounting for the orientation of the drone
combination_method = "sum"
normalization_method = "none"
#normalization_method = "sum_to_one"
#normalization_method = "zero_to_one"
#combination_method = "product"
beam_former.init_dynamic_estimate(combination_n=len(degrees), 
                                  combination_method=combination_method, 
                                  normalization_method=normalization_method)
for spectrum, degree in zip(spectra, degrees):
    beam_former.add_to_dynamic_estimates(spectrum, -degree)

combined_spectra = beam_former.get_dynamic_estimate()

In [ ]:
for spectrum, degree in zip(spectra, degrees):
    plot_spectrum(spectrum, from_0_to_360(gt_angle_deg - degree))
    plt.title(f'rot {degree}')

plot_spectrum(combined_spectra, from_0_to_360(gt_angle_deg - degrees[-1]))
plt.title(f'combined spectra')
pass

In [ ]:
# signals with delay correction

signals_f_list_delayed = []
for i, (sig_f, time) in enumerate(zip(signals_f_list, times_list)):
    exp_factor = np.exp(1j * 2 * np.pi * frequencies * time)
    print('phase difference in degrees:', from_0_to_2pi(np.angle(exp_factor))*180/np.pi)
    signals_f_delayed = np.multiply(sig_f, exp_factor[:, np.newaxis])  # frequencies x n_mics
    signals_f_list_delayed.append(signals_f_delayed)
    
for sig_f, signals_f_delayed, time in zip(signals_f_list, signals_f_list_delayed, times_list):
    print(f'original phases (time={time:.3f}):', from_0_to_2pi(np.angle(sig_f)))
    print(f'corrected phases(time={time:.3f}):', from_0_to_2pi(np.angle(signals_f_delayed)))
    
print('multi-mic:', from_0_to_2pi(np.angle(signals_f_multimic)))

# combine raw signals

In [ ]:
from algos_basics import get_mic_delays

beam_former = BeamFormer(mic_positions=mics_array)

signals_f_tot_delay = np.empty((len(frequencies), 0))
signals_f_tot_zero = np.empty((len(frequencies), 0))

for i, (sig_f, signals_f_delayed) in enumerate(zip(signals_f_list, signals_f_list_delayed)):
    signals_f_tot_delay = np.c_[signals_f_tot_delay, signals_f_delayed]
    # for testing purposes, we also look at the spectra without compensating
    # for the delays.
    signals_f_tot_zero = np.c_[signals_f_tot_zero, sig_f]

Rtot_zero = beam_former.get_correlation(signals_f_tot_zero)
combined_raw_zero = beam_former.get_mvdr_spectrum(Rtot_zero, frequencies)

Rtot_delay = beam_former.get_correlation(signals_f_tot_delay)
print(np.angle(Rtot_delay))
combined_raw_delay = beam_former.get_mvdr_spectrum(Rtot_delay, frequencies)

plot_spectrum(combined_raw_delay, degree=gt_angle_deg)
plt.title('combined raw with correct delay')

plot_spectrum(combined_raw_zero, degree=gt_angle_deg)
plt.title('combined raw with zero delay')
pass

# compare results

In [ ]:
chosen_idx = np.argmin(np.abs(frequencies - frequency_hz))
print(chosen_idx)
plt.figure()
plt.semilogy(angles, combined_spectra[chosen_idx], label='combined spectra')
plt.semilogy(angles, combined_raw_delay[chosen_idx], label='raw delay')
plt.semilogy(angles, combined_raw_zero[chosen_idx], label='raw zero', ls=':')
plt.semilogy(angles, spectrum_multimic[chosen_idx], label='multi-mic', ls=':')
plt.axvline(gt_angle_deg, color='C3', ls=':')
plt.axvline(from_0_to_360(gt_angle_deg - degrees[-1]), color='C0', ls=':')
plt.legend()

# Full pipeline